In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr, permutation_test
import sys
import argparse
import h5py
from scipy.stats import t as student_t
from statsmodels.stats import multitest as mt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
def deconfound_inputs(y):
    
    y = y - y.mean(axis=0)
    if np.sum(np.isnan(y)) == 0:
        beta = np.linalg.pinv(confounds).dot(y)
        beta[np.abs(beta) < 1e-10] = 0
        yd = y - confounds.dot(beta)
        yd = yd - yd.mean()
    else:
        print('ERROR! Encoundered a NaN! Function Needs Updating!')
    
    return yd

def apply_FDR_correction(p_values):
    '''
    Code addapted from Emma Bluemke & Nicola Dinsdale
    Wrapper for Benjamini/Hochberg (non-negative) p-value correction for multiple tests.
    '''
    p_values_corrected = mt.multipletests(p_values, alpha=0.05, method='fdr_bh')[1] 
    
    return p_values_corrected


def standardize_data(variables):
    '''
    Code addapted from Emma Bluemke & Nicola Dinsdale
    '''

    number_of_subjects=variables.shape[0]

    # Compute the arithmetic mean & std along the specified axis, ignoring NaNs.
    variables_mean_ignore_NaNs = np.nanmean(variables,axis=0)
    variables_std_ignore_NaNs = np.nanstd(variables,axis=0)

    # We standardize the data
    variables_scaled = variables - np.tile(variables_mean_ignore_NaNs,(number_of_subjects,1))
    variables_scaled = variables_scaled / np.tile(variables_std_ignore_NaNs,(number_of_subjects,1))

    # Calculate how many variables are non NaN
    number_of_non_NaN =np.sum(np.isnan(variables)==False,axis=0) #np.nanstd has N**0.5 in divisor

    return variables_scaled, number_of_non_NaN


def statistic(x, y):
    return pearsonr(x,y)[0]
    

def correlate_with_fdr_correction(deltas, ukb_variables):
    """
    Code addapted from Emma Bluemke & Nicola Dinsdale

    correlate age deltas with rows in ukb_variables
    returns pearson_r, t_test_statistic, p_values_corrected, p_values of length number_of_variables

    pearson_r = raw Pearson correlation value (between -1 and 1)
    t_test_statistic t test statistic for each variable ()
    p_values_corrected is corrected p-values values (fdr corrected)
    p_values is non corrected p-values
    """

    ukb_variables_scaled , ukb_variales_number_of_non_NaN = standardize_data(ukb_variables)
    ukb_variables_NaN_matrix = np.isnan(ukb_variables_scaled)
    ukb_variables_scaled[ukb_variables_NaN_matrix] = 0

    sys.stdout.flush()

    number_of_variables = ukb_variables_scaled.shape[1]

    deltas_array = np.tile(deltas,(number_of_variables,1)).T
    deltas_array[ukb_variables_NaN_matrix] = np.nan
    deltas_array = standardize_data(deltas_array)[0]
    deltas_array[ukb_variables_NaN_matrix] = 0

#     pearson_r = np.sum(deltas_array * ukb_variables_scaled, axis=0) / ukb_variales_number_of_non_NaN
    pearson_r = np.zeros(deltas_array.shape[1])
    p_values = np.zeros(deltas_array.shape[1])
    
    for idx in range(pearson_r.shape[0]):
        res = permutation_test(
            (deltas_array[:,idx], ukb_variables_scaled[:,idx]), 
            statistic, 
            permutation_type='pairings', 
            n_resamples=5000, 
            random_state=1
        )
        
        pearson_r[idx] = res.statistic
        p_values[idx] = res.pvalue
    
    
    t_test_statistic = pearson_r * ((ukb_variales_number_of_non_NaN-2)/(1-pearson_r**2))**0.5
    t_test_statistic = np.abs(t_test_statistic)
    t_test_statistic[ukb_variales_number_of_non_NaN <= 2] = np.nan #set those that had d.o.f 0 to nan

#     p_values=np.zeros(len(t_test_statistic))
    for j in range(len(t_test_statistic)):
        if t_test_statistic[j] is not np.nan:
#             p_values[j] = student_t.sf(t_test_statistic[j], ukb_variales_number_of_non_NaN[j] - 2) * 2
            pass
        else:
            p_values[j]=np.nan

    p_values_NaN_matrix = np.isnan(p_values)
    p_values_no_NaN = p_values[p_values_NaN_matrix==False]
    p_values_corrected_raw = apply_FDR_correction(p_values_no_NaN)

    p_values_corrected=np.zeros(len(p_values))
    p_values_corrected[p_values_NaN_matrix==False] = p_values_corrected_raw
    p_values_corrected[p_values_NaN_matrix==True] = np.nan

    return pearson_r, t_test_statistic, p_values_corrected, p_values

def align_subjects(subjects_df, subjects_h5, age_delta_decon, subjects_to_be_ignored):
    '''
    Here we make sure that the subjects match between the results and the nIDP/IDP dataframes
    The subjects_df uses slightly older data, and subjects might have left the study since it's generation
    If need be, this will be corrected at a later stage!
    '''
    
    if len(subjects_df) == len(subjects_h5):
        assert (subjects_df == subjects_h5).all()
    else:
        idx_elim = np.where(subjects_df == subjects_to_be_ignored)[0][0]
        subjects_df = np.delete(subjects_df, idx_elim)
        assert (subjects_df == subjects_h5).all()
        age_delta_decon = np.delete(age_delta_decon, idx_elim)
    return subjects_df, age_delta_decon

def correlations_and_plots(age_delta_decon, deconf_subset, names_subset, idxs, categories_subset, title, 
                           corr_flag='IDP'):
    
    corr_results = correlate_with_fdr_correction(age_delta_decon, deconf_subset)
    
    corr_df = pd.DataFrame.from_dict({
        'idx': idxs,
        "names": names_subset,
        'Categories': categories_subset,
        "pearson_r": corr_results[0],
        "t_test_statistic": corr_results[1],
        "p_values_corrected": corr_results[2],
        "p_values": corr_results[3],
        "abs_pearson_r": np.abs(corr_results[0]),
        "log_p_values": -np.log10(corr_results[3]),
        "log_p_values_corrected": -np.log10(corr_results[2]),
    })
    
    if corr_flag == 'IDP':
        short_p_values = np.extract(corr_df.p_values !=0, corr_df.p_values)
        bonferroni_threshold = 0.05/len(short_p_values) #... Bonferroni
    else:
        bonferroni_threshold = 0.05/len(corr_df.p_values) #... Bonferroni
    bonferroni_threshold = -np.log10(bonferroni_threshold)

    arguments = np.argsort(corr_df.p_values_corrected)
    sorted_p_values_corrected = corr_df.p_values_corrected[arguments]
    sorted_p_values = corr_df.p_values[arguments]
    if len(sorted_p_values_corrected[sorted_p_values_corrected<=0.05]):
        false_discovery_rate = np.nanmax(sorted_p_values[sorted_p_values_corrected<=0.05])
        false_discovery_rate = -np.log10(false_discovery_rate)
    else:

        false_discovery_rate = None
    
    return corr_df, bonferroni_threshold, false_discovery_rate

In [3]:
from scipy.stats import norm
rng = np.random.default_rng()
x = norm.rvs(size=6, random_state=rng)
y = norm.rvs(size=6, loc = 3, random_state=rng)
statistic = stats.pearsonr
statistic(x, y)

(-0.3932510402137002, 0.44053086470329966)

In [4]:
x = norm.rvs(size=6, random_state=rng)
y = norm.rvs(size=(2,6), loc = 3, random_state=rng)
y.shape

(2, 6)

In [5]:
res = stats.permutation_test((x, y), statistic,
                        n_resamples=np.inf, axis=1)

In [6]:
print(res.statistic[0])

[-0.09598688  0.85646187]


In [7]:
print(res.pvalue[0])

[0.01298701 0.11688312]


In [8]:
res = stats.permutation_test((x, y[1]), statistic,
                        n_resamples=np.inf)
print(res.statistic)

[-0.06086362  0.9088173 ]


In [9]:
pearsonr = stats.pearsonr

from datetime import datetime

def statistic(x, y):
    return pearsonr(x,y)[0]


from scipy.stats import norm
rng = np.random.default_rng()
# statistic = stats.pearsonr

x = norm.rvs(size=5000, random_state=rng)
y = norm.rvs(size=5000, loc=0.3, random_state=rng)
print(statistic(x, y))

t0=datetime.now()
for i in range(100):
    res = stats.permutation_test((x, y), statistic, permutation_type='pairings', n_resamples=5000, random_state=1)
t1=datetime.now()
print(t1 - t0, (t1-t0)/100)
print(res.statistic)
print(res.pvalue)

-0.01182135520077738


KeyboardInterrupt: 

In [10]:
subjects_to_be_ignored = [21269692]
subject_sex = 'female'
prototype_flag = True

deconfound_flag = True
vars_cutoff_nan = 0.1

In [11]:
# Identify Correct File to Load

IDP_file = '../Analsysis Female New/female_test_IDPs.h5'
confounds_file = '../Analsysis Female New/female_test_conf12.h5'
data_file = 'female_test.pkl'

confounds = h5py.File(confounds_file, 'r')
confounds = confounds['conf12'][()]

IDP_h5 = h5py.File(IDP_file, 'r')
subjects_h5 = IDP_h5['subjects'][()]

# Load the relevant dataframe based on the required modality

df = pd.read_pickle(data_file)

IDP_h5 = h5py.File(IDP_file, 'r')
vars_i_deconf = IDP_h5['vars_i_deconf'][()]
subjects_h5 = IDP_h5['subjects'][()]

with open('../Analsysis Female New/varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
with open('../Analsysis Female New/vars_categories.txt') as f:
    vars_categories = f.readlines()
    vars_categories = [l.strip('\n\r') for l in vars_categories]
    vars_categories = np.array(vars_categories)

varsIDX = np.arange(0, len(varsHeader), 1).astype(int)

vars_perc_not_nan = np.sum(np.isnan(vars_i_deconf)==False,axis=0) / vars_i_deconf.shape[0]
vars_idx_nan_cutoff = np.where(vars_perc_not_nan >= vars_cutoff_nan)[0]

varsHeader_subset = varsHeader[vars_idx_nan_cutoff]
vars_i_deconf_subset = vars_i_deconf[:, vars_idx_nan_cutoff]
vars_categories_subset = vars_categories[vars_idx_nan_cutoff]
varsIDX_subset = varsIDX[vars_idx_nan_cutoff]

del IDP_h5

In [65]:
vars_corrs = []
vars_mods = []
vars_bnf_thrs = []
vars_fdr_thrs = []

for idx in range(len(df)):
    df_mod = df.iloc[idx].dataframe
    modality = df.iloc[idx].modality
    age_delta_decon = df_mod.age_delta_decon.to_numpy()
    subjects = df_mod['Unnamed: 0'].to_numpy()
    subjects, age_delta_decon = align_subjects(subjects, subjects_h5, 
                                                  age_delta_decon, subjects_to_be_ignored)
    
    if deconfound_flag == True:
        age_delta_decon = deconfound_inputs(age_delta_decon)
        
        
    corr_df, bonferroni_threshold, false_discovery_rate = correlations_and_plots(
                                    age_delta_decon = age_delta_decon, deconf_subset = vars_i_deconf_subset, 
                                    names_subset = varsHeader_subset, idxs = varsIDX_subset,
                                    categories_subset = vars_categories_subset,
                                    title = modality, corr_flag='vars')
    vars_mods.append(modality)
    vars_corrs.append(corr_df)
    vars_bnf_thrs.append(bonferroni_threshold)
    vars_fdr_thrs.append(false_discovery_rate)
    
    if idx==3:
        break
    
vars_df = pd.DataFrame.from_dict({
    'modality': vars_mods,
    'bonf': vars_bnf_thrs,
    'fdr': vars_fdr_thrs,
    'dataframe': vars_corrs
})

In [ ]:
vars_df.to_pickle('female_vars_permutation_tested.pkl', protocol = 4)

In [13]:
idx = 0

df_mod = df.iloc[idx].dataframe
modality = df.iloc[idx].modality
age_delta_decon = df_mod.age_delta_decon.to_numpy()
subjects = df_mod['Unnamed: 0'].to_numpy()
subjects, age_delta_decon = align_subjects(subjects, subjects_h5, 
                                              age_delta_decon, subjects_to_be_ignored)

if deconfound_flag == True:
    age_delta_decon = deconfound_inputs(age_delta_decon)

In [14]:
deltas = np.copy(age_delta_decon)
ukb_variables = vars_i_deconf_subset

In [15]:
type(pearson_r)

NameError: name 'pearson_r' is not defined

In [16]:
ukb_variables.shape

(5021, 13809)

In [17]:
ukb_variables_scaled , ukb_variales_number_of_non_NaN = standardize_data(ukb_variables)
ukb_variables_NaN_matrix = np.isnan(ukb_variables_scaled)
ukb_variables_scaled[ukb_variables_NaN_matrix] = 0

sys.stdout.flush()

number_of_variables = ukb_variables_scaled.shape[1]

deltas_array = np.tile(deltas,(number_of_variables,1)).T
deltas_array[ukb_variables_NaN_matrix] = np.nan
deltas_array = standardize_data(deltas_array)[0]
deltas_array[ukb_variables_NaN_matrix] = 0

pearson_r = np.sum(deltas_array * ukb_variables_scaled, axis=0) / ukb_variales_number_of_non_NaN

In [145]:
pearsonr(deltas_array[:,2], ukb_variables_scaled[:,2])[0]

-0.05016419856295517

In [143]:
pearson_r[0:3]

array([-1.19415069e-02, -9.71047499e-05, -5.01641986e-02])

In [147]:
pearson_r.shape

(13809,)

In [149]:
deltas_array.shape[1]

13809

In [150]:
np.empty(deltas_array.shape[1]).shape

(13809,)

In [151]:
deltas_array[:,0].shape

(5021,)

In [117]:
deltas_array = np.tile(deltas,(number_of_variables,1)).T
deltas_array[ukb_variables_NaN_matrix] = np.nan

variables=deltas_array

print(variables)
print('------')

number_of_subjects=variables.shape[0]

# Compute the arithmetic mean & std along the specified axis, ignoring NaNs.
variables_mean_ignore_NaNs = np.nanmean(variables,axis=0)
variables_std_ignore_NaNs = np.nanstd(variables,axis=0)

print(variables_mean_ignore_NaNs)

# We standardize the data
variables_scaled = variables - np.tile(variables_mean_ignore_NaNs,(number_of_subjects,1))
variables_scaled = variables_scaled / np.tile(variables_std_ignore_NaNs,(number_of_subjects,1))

# Calculate how many variables are non NaN
number_of_non_NaN =np.sum(np.isnan(variables)==False,axis=0) #np.nanstd has N**0.5 in divisor

[[ 0.02383519         nan         nan ...         nan  0.02383519
   0.02383519]
 [-0.26473594         nan         nan ...         nan -0.26473594
  -0.26473594]
 [ 0.67400042         nan         nan ...         nan  0.67400042
   0.67400042]
 ...
 [-2.27160436         nan         nan ...         nan         nan
          nan]
 [ 2.55569929         nan         nan ...         nan         nan
          nan]
 [ 0.913351           nan  0.913351   ...         nan  0.913351
   0.913351  ]]
------
[-0.00034379 -0.11382293 -0.02613652 ... -0.07509479 -0.03095923
 -0.03095923]


In [152]:
len(pearson_r)

13809

In [153]:
np.__version__

'1.21.6'